In [1]:
import os
import signal
import sys
import time
import casadi as ca

import pinocchio as pin
from pinocchio import casadi as cpin
import numpy as np

from pinocchio.robot_wrapper import RobotWrapper
from utils.meshcat_viewer_wrapper import MeshcatVisualizer

In [2]:
# load go2 robot
modelPath = "/home/dong/humanrobot/quadruped/robot/"
URDF_FILENAME = "go2_description.urdf"

Robot_Go2 = RobotWrapper.BuildFromURDF(modelPath + URDF_FILENAME, modelPath,
                pin.JointModelFreeFlyer()) # Load URDF file
model = Robot_Go2.model
data = model.createData()

lfFoot, rfFoot, lhFoot, rhFoot = "FL_foot", "FR_foot", "RL_foot", "RR_foot"

lfFoot_ID = model.getFrameId(lfFoot)
rfFoot_ID = model.getFrameId(rfFoot)
lhFoot_ID = model.getFrameId(lhFoot)
rhFoot_ID = model.getFrameId(rhFoot)

# display = crocoddyl.MeshcatDisplay(
#     Robot_Go2, frameNames=[lfFoot, rfFoot, lhFoot, rhFoot])

q0 = pin.utils.zero(model.nq)
q0[2] = 0.325 # z
q0[6] = 1  # q.w 7+12
q0[7:] = [0., np.pi/4, -np.pi/2, 0., np.pi/4, -np.pi/2, 0.,np.pi/4,-np.pi/2, 0.,np.pi/4,-np.pi/2]


v0 = pin.utils.zero(Robot_Go2.model.nv)
x0 = np.concatenate([q0, v0])
# display.display(x0)

display = MeshcatVisualizer(Robot_Go2)
display.display(q0)
time.sleep(0.05)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


In [3]:
pin.framesForwardKinematics(model, data, Robot_Go2.q0)
pin.computeTotalMass(model, data)
pin.updateFramePlacements(model, data)

mass = data.mass[0]
nq = model.nq
nv = model.nv

cmodel = cpin.Model(model)
cdata = cmodel.createData()
cq = ca.SX.sym("q", nq, 1)
cdq = ca.SX.sym("dq", nv, 1)
cddq = ca.SX.sym("ddq", nv, 1)

cf_left_front = ca.SX.sym("f_left_front", 6, 1)
cf_left_back = ca.SX.sym("f_left_back", 6, 1)
cf_right_front = ca.SX.sym("f_right_front", 6, 1)
cf_right_back = ca.SX.sym("f_right_back", 6, 1)

ctau = ca.SX.sym("tau", nv, 1)

cpin.centerOfMass(cmodel, cdata, cq, cdq, False)
cpin.forwardKinematics(cmodel, cdata, cq, cdq, cddq)
cpin.computeCentroidalMap(cmodel, cdata, cq)
cpin.computeCentroidalMapTimeVariation(cmodel, cdata, cq, cdq)
cpin.updateFramePlacements(cmodel, cdata)

cpin.computeJointJacobians(cmodel, cdata, cq)
cpin.computeJointJacobiansTimeVariation(cmodel, cdata, cq, cdq)


SX(@1=2, @2=(@1*q_4), @3=(@2*q_3), @4=(@1*q_5), @5=(@4*q_6), @6=(@3+@5), @7=1, @8=(@1*q_3), @9=(@8*q_3), @10=(@4*q_5), @11=(@7-(@9+@10)), @12=(@4*q_4), @13=(@8*q_6), @14=(@12-@13), @15=((@6*dq_3)+((@11*dq_4)+(@14*dq_5))), @16=(@4*q_3), @17=(@2*q_6), @18=(@16-@17), @19=(@12+@13), @20=(@2*q_4), @21=(@7-(@9+@20)), @22=((@18*dq_3)+((@19*dq_4)+(@21*dq_5))), @23=(@7-(@20+@10)), @24=(@3-@5), @25=(@16+@17), @26=((@23*dq_3)+((@24*dq_4)+(@25*dq_5))), @27=0, @28=(((@6*dq_0)+((@11*dq_1)+(@14*dq_2)))+((q_2*@26)-(q_0*@22))), @29=(((@18*dq_0)+((@19*dq_1)+(@21*dq_2)))+((q_0*@15)-(q_1*@26))), @30=((q_0*@6)-(q_1*@23)), @31=((q_2*@23)-(q_0*@18)), @32=(((@23*dq_0)+((@24*dq_1)+(@25*dq_2)))+((q_1*@22)-(q_2*@15))), @33=((q_1*@18)-(q_2*@6)), @34=((q_0*@11)-(q_1*@24)), @35=((q_2*@24)-(q_0*@19)), @36=((q_1*@19)-(q_2*@11)), @37=((q_0*@14)-(q_1*@25)), @38=((q_2*@25)-(q_0*@21)), @39=((q_1*@21)-(q_2*@14)), @40=0.0465, @41=(dq_0-(@40*dq_5)), @42=cos(q_7), @43=sin(q_7), @44=((@11*@42)+(@14*@43)), @45=0.1934, @46=(dq_

In [4]:
# get CMM and time derivative
Ag = cdata.Ag
dAg = cdata.dAg

J = cdata.J
Jdot = cdata.dJ

J1 = cpin.computeFrameJacobian(cmodel, cdata, cq, lfFoot_ID, pin.LOCAL)
J2 = cpin.computeFrameJacobian(cmodel, cdata, cq, rfFoot_ID, pin.LOCAL)
J3 = cpin.computeFrameJacobian(cmodel, cdata, cq, lhFoot_ID, pin.LOCAL)
J4 = cpin.computeFrameJacobian(cmodel, cdata, cq, rhFoot_ID, pin.LOCAL)

J1dot = cpin.frameJacobianTimeVariation(cmodel, cdata, cq, cdq, lfFoot_ID, pin.LOCAL)
J2dot = cpin.frameJacobianTimeVariation(cmodel, cdata, cq, cdq, rfFoot_ID, pin.LOCAL)
J3dot = cpin.frameJacobianTimeVariation(cmodel, cdata, cq, cdq, lhFoot_ID, pin.LOCAL)
J4dot = cpin.frameJacobianTimeVariation(cmodel, cdata, cq, cdq, rhFoot_ID, pin.LOCAL)

acc1 = cpin.getFrameAcceleration(cmodel, cdata, lfFoot_ID, pin.LOCAL).vector
acc2 = cpin.getFrameAcceleration(cmodel, cdata, rfFoot_ID, pin.LOCAL).vector
acc3 = cpin.getFrameAcceleration(cmodel, cdata, lhFoot_ID, pin.LOCAL).vector
acc4 = cpin.getFrameAcceleration(cmodel, cdata, rhFoot_ID, pin.LOCAL).vector

vel1 = cpin.getFrameVelocity(cmodel, cdata, lfFoot_ID, pin.LOCAL).vector
vel2 = cpin.getFrameVelocity(cmodel, cdata, rfFoot_ID, pin.LOCAL).vector
vel3 = cpin.getFrameVelocity(cmodel, cdata, lhFoot_ID, pin.LOCAL).vector
vel4 = cpin.getFrameVelocity(cmodel, cdata, rhFoot_ID, pin.LOCAL).vector

cpin.crba(cmodel, cdata, cq)
cpin.computeCoriolisMatrix(cmodel, cdata, cq, cdq)
cpin.computeGeneralizedGravity(cmodel, cdata, cq)

SX(@1=6.923, @2=9.81, @3=2, @4=(@3*q_5), @5=(@3*q_4), @6=(@2*((@4*q_3)-(@5*q_6))), @7=(@1*@6), @8=0.678, @9=(@8*@6), @10=cos(q_17), @11=1.152, @12=sin(q_17), @13=cos(q_16), @14=(@3*q_3), @15=(@2*(1-((@14*q_3)+(@5*q_4)))), @16=sin(q_16), @17=(@2*((@4*q_4)+(@14*q_6))), @18=((@13*@15)-(@16*@17)), @19=((@10*@6)-(@12*@18)), @20=(@11*@19), @21=cos(q_18), @22=0.194, @23=sin(q_18), @24=((@12*@6)+(@10*@18)), @25=(@22*((@21*@19)-(@23*@24))), @26=(@22*((@23*@19)+(@21*@24))), @27=((@21*@25)+(@23*@26)), @28=(@20+@27), @29=(@11*@24), @30=(@29+((@21*@26)-(@23*@25))), @31=((@10*@28)+(@12*@30)), @32=(@9+@31), @33=(@8*@6), @34=cos(q_14), @35=sin(q_14), @36=cos(q_13), @37=sin(q_13), @38=((@36*@15)-(@37*@17)), @39=((@34*@6)-(@35*@38)), @40=(@11*@39), @41=cos(q_15), @42=sin(q_15), @43=((@35*@6)+(@34*@38)), @44=(@22*((@41*@39)-(@42*@43))), @45=(@22*((@42*@39)+(@41*@43))), @46=((@41*@44)+(@42*@45)), @47=(@40+@46), @48=(@11*@43), @49=(@48+((@41*@45)-(@42*@44))), @50=((@34*@47)+(@35*@49)), @51=(@33+@50), @52=(

In [6]:
Msym = cdata.M  # 质量矩阵
Csym = ca.mtimes(cdata.C, cdq) # 科里奥利力
gsym = cdata.g 
S = np.zeros((nv, model.nv))
S[6:, 6:] = np.eye(nv - 6) # 选择矩阵，通常用于浮动基座的机器人
# print("M:\n", Msym)
# print("C:\n", Csym)
# print("g:\n", gsym)

A = ca.vertcat(ca.horzcat(Msym, -S.T, -J1.T, -J2.T, -J3.T, -J4.T))
b = -Csym - gsym

cdynamics = ca.Function(
        "dynamics",
        [  cq, cdq, cddq, ctau, cf_left_front, cf_right_front, cf_left_back, cf_right_back, ], 
        [  ca.mtimes(A, ca.vertcat(cddq, ctau, cf_left_front, cf_right_front, cf_left_back, cf_right_back, )) - b],)

ccom = ca.Function("com", [cq], [cdata.com[0]])
cvcom = ca.Function("vcom", [cq, cdq], [cdata.vcom[0]])


In [7]:
# Velocity and acceleration constraints
vel1_constraint = ca.Function(
    "vel_constraint",
    [cq, cdq],
    [
        ca.vertcat(
            cpin.getFrameVelocity(cmodel, cdata, lfFoot_ID, pin.LOCAL).vector,
        )
    ],
)

vel2_constraint = ca.Function(
    "vel_constraint",
    [cq, cdq],
    [
        ca.vertcat(
            cpin.getFrameVelocity(cmodel, cdata, rfFoot_ID, pin.LOCAL).vector,
        )
    ],
)

vel3_constraint = ca.Function(
    "vel_constraint",
    [cq, cdq],
    [
        ca.vertcat(
            cpin.getFrameVelocity(cmodel, cdata, lhFoot_ID, pin.LOCAL).vector,
        )
    ],
)

vel4_constraint = ca.Function(
    "vel_constraint",
    [cq, cdq],
    [
        ca.vertcat(
            cpin.getFrameVelocity(cmodel, cdata, rhFoot_ID, pin.LOCAL).vector,
        )
    ],
)

acc1_constraint = ca.Function(
    "acc1_constraint",
    [cq, cdq, cddq],
    [
        ca.vertcat(
            cpin.getFrameAcceleration(cmodel, cdata, lfFoot_ID, pin.LOCAL).vector,
        )
    ],
)

acc2_constraint = ca.Function(
    "acc2_constraint",
    [cq, cdq, cddq],
    [
        ca.vertcat(
            cpin.getFrameAcceleration(cmodel, cdata, rfFoot_ID, pin.LOCAL).vector,
        )
    ],
)
acc3_constraint = ca.Function(
    "acc3_constraint",
    [cq, cdq, cddq],
    [
        ca.vertcat(
            cpin.getFrameAcceleration(cmodel, cdata, lhFoot_ID, pin.LOCAL).vector,
        )
    ],
)
acc4_constraint = ca.Function(
    "acc4_constraint",
    [cq, cdq, cddq],
    [
        ca.vertcat(
            cpin.getFrameAcceleration(cmodel, cdata, rhFoot_ID, pin.LOCAL).vector,
        )
    ],
)

# Jacobian constraint function
jac_cost = ca.Function(
    "jac_cost",
    [cq, cdq, cddq],
    [ca.vertcat(ca.mtimes(J1, cddq) + ca.mtimes(J1dot, cdq))],
)

# Angular and linear momentum rate of change
Hdot_fun = ca.Function(
    "ang_mom_dot",
    [cq, cdq, cddq],
    [ca.mtimes(Ag[:3, :], cddq) + ca.mtimes(dAg[:3, :], cdq)],
)
Ldot_fun = ca.Function(
    "lin_mom_dot",
    [cq, cdq, cddq],
    [ca.mtimes(Ag[3:, :], cddq) + ca.mtimes(dAg[3:, :], cdq)],
)

In [8]:
# Mass, Coriolis, and gravity matrices
getM = ca.Function("M", [cq], [ca.vertcat(Msym)])
getC = ca.Function("C", [cq, cdq], [ca.vertcat(Csym)])
getG = ca.Function("g", [cq], [ca.vertcat(gsym)])

In [9]:
# Static torque calculation
getStaticTorque = ca.Function(
    "static_tau",
    [cq, cdq, cf_left_front, cf_right_front, cf_left_back, cf_right_back],
    [ca.vertcat(gsym - ca.mtimes(ca.transpose(J1), cf_left_front + cf_right_front + cf_left_back + cf_right_back))],
)

In [10]:
# Friction constraint
mu = 0.7
num_contacts = 4
A_i = np.asarray(
    [
        [0, 0, 0, 1, 0, -mu],
        [0, 0, 0, -1, 0, -mu],
        [0, 0, 0, 0, 1, -mu],
        [0, 0, 0, 0, -1, -mu],
    ]
)
fric_constraint = ca.Function(
    "fric_constraint",
    [cf_left_front, cf_right_front, cf_left_back, cf_right_back],
    [
        ca.vertcat(
            ca.mtimes(
                np.kron(np.eye(num_contacts), A_i), ca.vertcat(cf_left_front, cf_right_front, cf_left_back, cf_right_back)
            )
        )
    ],
)
com_ref = ccom(q0)
vcom_ref = cvcom(q0, v0)

In [11]:
# Tuning variables
d_bougemarte = 2e2 
k_lin = 1e0
k_qdd_b = 1e1
k_qdd_j = 1e0
d_qdd_b = 1e2
d_qdd_j = 1e-2
d_lin = 1e-2
w_body = 1e0
w_joints = 1e0
w_angmom = 1e0
w_linmom = 1e0
beta_body = 5e2
beta_joints = 1e0
beta_angmom = 1e0
beta_linmom = 1e0
        
i = 0

In [38]:
# Generate reasonable test data
def generate_test_data():
    p = np.array([0, 0, 0.325])
    o = np.array([0, 0, 0, 1])
    vl = np.array([0, 0, 0])
    al = np.array([0, 0, 0])
    xc = np.zeros(12)
    vc = np.zeros(12)
    q = np.vstack((np.reshape(np.concatenate((p, o)), (7, 1)), xc.reshape(-1, 1)))
    v = np.vstack((np.reshape(np.concatenate((vl, al)), (6, 1)), vc.reshape(-1, 1)))
    return q, v

i = 0

q, v = generate_test_data()

pin.forwardKinematics(model, data, q, v)
differ = pin.difference(model, q0, q)
qdd_b_des = k_qdd_b * np.reshape(differ[:6], (6, 1)) + d_qdd_b * (v0.reshape(18,1)[:6]  - v[:6])
qdd_j_des = k_qdd_j * (q0.reshape(19,1)[7:] - q[7:]) + d_qdd_j * (v0.reshape(18,1)[6:] - v[6:])
Ldot_des = k_lin * mass * (com_ref - ccom(q)) - d_lin * mass * (vcom_ref - cvcom(q, v))
Hdot_des = np.zeros((3, 1))


In [39]:

opti = ca.Opti("conic")
var_ddq = opti.variable(nv)
var_tau = opti.variable(12)
var_f_left_front = opti.variable(6)
var_f_right_front = opti.variable(6)
var_f_left_back = opti.variable(6)
var_f_right_back = opti.variable(6)

objective = 0
objective = objective + (
    beta_body * ca.sumsqr(w_body * (qdd_b_des - var_ddq[:6]))
    + beta_joints * ca.sumsqr(w_joints * (qdd_j_des - var_ddq[6:]))
    + beta_angmom * ca.sumsqr(w_angmom * (Hdot_des - Hdot_fun(q, v, var_ddq)))
    + beta_linmom * ca.sumsqr(w_linmom * (Ldot_des - Ldot_fun(q, v, var_ddq)))
    + ca.sumsqr(var_tau))
opti.subject_to(
        acc1_constraint(q, v, var_ddq)
        == -2.0
        * np.sqrt(d_bougemarte)
        * pin.getFrameVelocity(model, data, lfFoot_ID, pin.LOCAL).vector
    )
opti.subject_to(
        acc2_constraint(q, v, var_ddq)
        == -2.0
        * np.sqrt(d_bougemarte)
        * pin.getFrameVelocity(model, data, rfFoot_ID, pin.LOCAL).vector
    )
opti.subject_to(
        acc3_constraint(q, v, var_ddq)
        == -2.0
        * np.sqrt(d_bougemarte)
        * pin.getFrameVelocity(model, data, lhFoot_ID, pin.LOCAL).vector
    )
opti.subject_to(
        acc4_constraint(q, v, var_ddq)
        == -2.0
        * np.sqrt(d_bougemarte)
        * pin.getFrameVelocity(model, data, rhFoot_ID, pin.LOCAL).vector
    )
opti.subject_to(
        cdynamics(
            q,
            v,
            var_ddq,
            ca.vertcat(np.zeros((6, 1)), var_tau),
            var_f_left_front,
            var_f_right_front,
            var_f_left_back,
            var_f_right_back,
        ) == 0
    )
opti.subject_to(var_f_left_front[5] >= 0)
opti.subject_to(var_f_right_front[5] >= 0)
opti.subject_to(var_f_left_back[5] >= 0)
opti.subject_to(var_f_right_back[5] >= 0)

f_guess = ca.vertcat(0, 0, 0, 0, 0, data.mass[0] * 9.81 / 4.0)
tau_guess = getStaticTorque(q, v, f_guess, f_guess, f_guess, f_guess)
opti.set_initial(var_f_left_front, f_guess)
opti.set_initial(var_f_right_front, f_guess)
opti.set_initial(var_f_left_back, f_guess)
opti.set_initial(var_f_right_back, f_guess)
opti.set_initial(var_tau, tau_guess[6:])
opti.bounded(-300, var_tau, 300)
if i > 0:
    opti.set_initial(opti.x, prev_x)
opti.minimize(objective)
opti.solver("osqp", {"verbose": True})
sol = opti.solve_limited()
prev_x = opti.value(opti.x)
optimized_tau = opti.value(var_tau)
optimized_f_left_front = opti.value(var_f_left_front)
optimized_f_left_back = opti.value(var_f_left_back)
optimized_f_right_front = opti.value(var_f_right_front)
optimized_f_right_back = opti.value(var_f_right_back)

print("Optimized Tau: ", optimized_tau)
print("Optimized f_left_front: ", optimized_f_left_front)
print("Optimized f_left_back: ", optimized_f_left_back)
print("Optimized f_right_front: ", optimized_f_right_front)
print("Optimized f_right_back: ", optimized_f_right_back)



CasADi - 2024-07-23 15:12:17 MESSAGE("solver_qpsol::init") [.../casadi/core/function_internal.cpp:547]
-----------------------------------------------------------------
           OSQP v0.6.3  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2021
-----------------------------------------------------------------
problem:  variables n = 54, constraints m = 100
          nnz(P) + nnz(A) = 729
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: off, polish: off, time_limit: off

-----------------------------------------------------------------
           OSQP v0.6.3  -  Operator Splitting QP Solver
    

ValueError: cannot reshape array of size 36 into shape (6,1)